In [ ]:
!pip install lightgbm

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from dacon_law_class import mean_pooling, text_processor
import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_wine
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from xgboost.sklearn import XGBClassifier as xgb
# import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

 ___________________________
|                           |
|======== YearDream ========|
|===========================|
|==== DLC Well Imported ====|
|===========================|
|========= BYJASON =========|
|___________________________|



In [5]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
sample_submission = pd.read_csv('../sample_submission.csv')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [6]:
train

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [7]:
test

,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."
...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu..."


In [8]:
sample_submission

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [9]:
train.info()
print('\n\n\n')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2478 non-null   object
 1   first_party         2478 non-null   object
 2   second_party        2478 non-null   object
 3   facts               2478 non-null   object
 4   first_party_winner  2478 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 96.9+ KB




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            1240 non-null   object
 1   first_party   1240 non-null   object
 2   second_party  1240 non-null   object
 3   facts         1240 non-null   object
dtypes: object(4)
memory usage: 38.9+ KB


In [10]:
df = pd.DataFrame(train['facts'])
df

,facts
0,"On June 27, 1962, Phil St. Amant, a candidate ..."
1,Ramon Nelson was riding his bike when he suffe...
2,An Alabama state court convicted Billy Joe Mag...
3,Victor Linkletter was convicted in state court...
4,"On April 24, 1953 in Selma, Alabama, an intrud..."
...,...
2473,Congress amended the Clean Air Act through the...
2474,"Alliance Bond Fund, Inc., an investment fund, ..."
2475,"In 1992, the District Court sentenced Manuel D..."
2476,"On March 8, 1996, Enrico St. Cyr, a lawful per..."


In [11]:
temp = []
for i in range(len(df)):
    temp.append(text_processor(df['facts'][i]))

temp_dict = {"facts" : temp}
    
df = pd.DataFrame(temp_dict)
df

,facts
0,phil amant caidate for public office made tele...
1,ramon nelson was ridin his bike when he suffer...
2,an alabama ate court convicted billy joe mawoo...
3,victor linkletter was convicted in ate court o...
4,in selma alabama an intruder broke into apartm...
...,...
2473,conress ameed clean air act rouh enery policy ...
2474,alliance bo fu inc an invement fu purchased ap...
2475,in dirict court sentenced manuel peuero to mon...
2476,enrico cyr lawful permanent resident pled uilt...


In [12]:
df = pd.concat([train['ID'], df],axis=1)
temp = train.drop(columns=['facts', 'first_party_winner'], axis=1)
temp = pd.merge(temp, df, how='right', on='ID')
temptemp = train.drop(columns=['first_party', 'second_party', 'facts'], axis=1)
train_cleansed = pd.merge(temp, temptemp, how='right', on='ID')

train_cleansed

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,phil amant caidate for public office made tele...,1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,ramon nelson was ridin his bike when he suffer...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",an alabama ate court convicted billy joe mawoo...,1
3,TRAIN_0003,Linkletter,Walker,victor linkletter was convicted in ate court o...,0
4,TRAIN_0004,William Earl Fikes,Alabama,in selma alabama an intruder broke into apartm...,1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",conress ameed clean air act rouh enery policy ...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.",alliance bo fu inc an invement fu purchased ap...,1
2475,TRAIN_2475,Peguero,United States,in dirict court sentenced manuel peuero to mon...,0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,enrico cyr lawful permanent resident pled uilt...,0


In [13]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

combined_se_df = pd.DataFrame()
for fact in tqdm(train_cleansed['facts']):
    encoded_input = tokenizer(fact, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    se_df = pd.DataFrame(sentence_embeddings)
    combined_se_df = pd.concat([combined_se_df, se_df])
    
print(combined_se_df)

100%|████████████████████████| 2478/2478 [01:08<00:00, 35.95it/s]

         0         1         2         3         4         5         6    \
0  -0.084586  0.044809 -0.016432 -0.109756  0.018203  0.063628  0.055155   
0  -0.085407  0.099032 -0.077048 -0.005445  0.068653  0.056232  0.091459   
0  -0.069703  0.010037 -0.029463 -0.048004 -0.063775  0.062738  0.036750   
0  -0.035767  0.065843  0.041454 -0.048678  0.031218  0.120635  0.031755   
0  -0.094300  0.045971  0.015676 -0.040077  0.018894  0.038688  0.055605   
..       ...       ...       ...       ...       ...       ...       ...   
0  -0.069269  0.044468  0.045988  0.007961  0.078114  0.031617  0.008722   
0  -0.090639  0.025244 -0.054296 -0.027282 -0.086140 -0.005094  0.046781   
0  -0.044806  0.075822 -0.002418 -0.047819 -0.018801  0.059968  0.101083   
0  -0.069220  0.017232  0.005789 -0.042474 -0.035015  0.026547  0.039012   
0  -0.064640 -0.057630 -0.027339 -0.123645 -0.038989  0.019696  0.059252   

         7         8         9    ...       374       375       376       377  \
0   0.

In [14]:
facts_embeddings = combined_se_df.set_index(train['ID'])
facts_embeddings


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
ID,,,,,,,,,,,,,,,,,,,,,
TRAIN_0000,-0.084586,0.044809,-0.016432,-0.109756,0.018203,0.063628,0.055155,0.000317,0.030852,0.028107,...,0.069664,-0.007974,0.093355,0.047597,-0.046467,-0.066715,0.029723,0.009452,0.024468,0.021819
TRAIN_0001,-0.085407,0.099032,-0.077048,-0.005445,0.068653,0.056232,0.091459,0.056151,-0.055245,0.040278,...,0.026423,0.006050,0.015782,-0.016330,-0.128399,-0.024154,0.038394,-0.024557,0.060202,-0.042300
TRAIN_0002,-0.069703,0.010037,-0.029463,-0.048004,-0.063775,0.062738,0.036750,-0.026244,0.033911,0.013960,...,0.063877,-0.017928,0.096242,-0.003385,-0.026122,-0.077431,0.094061,-0.008745,0.069056,0.009244
TRAIN_0003,-0.035767,0.065843,0.041454,-0.048678,0.031218,0.120635,0.031755,0.038023,-0.057590,0.022954,...,0.043817,0.023443,0.108661,-0.005077,-0.098669,-0.053012,0.061100,0.070778,0.029561,-0.009112
TRAIN_0004,-0.094300,0.045971,0.015676,-0.040077,0.018894,0.038688,0.055605,0.019271,0.023044,0.069166,...,0.114352,0.031166,0.041276,0.064298,-0.047091,-0.024432,0.014282,0.054617,0.051875,0.014271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_2473,-0.069269,0.044468,0.045988,0.007961,0.078114,0.031617,0.008722,-0.009833,-0.086068,-0.025963,...,0.003805,-0.013601,-0.026617,0.026741,-0.077526,-0.032749,0.094584,0.004847,0.043765,0.001743
TRAIN_2474,-0.090639,0.025244,-0.054296,-0.027282,-0.086140,-0.005094,0.046781,0.043694,0.041948,-0.026018,...,-0.007459,-0.054953,0.020888,0.070636,-0.012312,-0.032366,0.027642,-0.049127,0.093051,-0.049553
TRAIN_2475,-0.044806,0.075822,-0.002418,-0.047819,-0.018801,0.059968,0.101083,0.114109,0.037637,0.055298,...,0.093023,0.041787,0.055687,-0.010128,-0.056995,0.019361,0.010142,0.035674,0.057293,0.014798


In [15]:
# X = pd.merge(facts_embeddings, first_party_weights, how='right', on='ID')
# X = pd.merge(X, second_party_weights, how='right', on='ID')
# X = facts_embeddings.drop(columns=['ID'])
X = facts_embeddings
X

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
ID,,,,,,,,,,,,,,,,,,,,,
TRAIN_0000,-0.084586,0.044809,-0.016432,-0.109756,0.018203,0.063628,0.055155,0.000317,0.030852,0.028107,...,0.069664,-0.007974,0.093355,0.047597,-0.046467,-0.066715,0.029723,0.009452,0.024468,0.021819
TRAIN_0001,-0.085407,0.099032,-0.077048,-0.005445,0.068653,0.056232,0.091459,0.056151,-0.055245,0.040278,...,0.026423,0.006050,0.015782,-0.016330,-0.128399,-0.024154,0.038394,-0.024557,0.060202,-0.042300
TRAIN_0002,-0.069703,0.010037,-0.029463,-0.048004,-0.063775,0.062738,0.036750,-0.026244,0.033911,0.013960,...,0.063877,-0.017928,0.096242,-0.003385,-0.026122,-0.077431,0.094061,-0.008745,0.069056,0.009244
TRAIN_0003,-0.035767,0.065843,0.041454,-0.048678,0.031218,0.120635,0.031755,0.038023,-0.057590,0.022954,...,0.043817,0.023443,0.108661,-0.005077,-0.098669,-0.053012,0.061100,0.070778,0.029561,-0.009112
TRAIN_0004,-0.094300,0.045971,0.015676,-0.040077,0.018894,0.038688,0.055605,0.019271,0.023044,0.069166,...,0.114352,0.031166,0.041276,0.064298,-0.047091,-0.024432,0.014282,0.054617,0.051875,0.014271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_2473,-0.069269,0.044468,0.045988,0.007961,0.078114,0.031617,0.008722,-0.009833,-0.086068,-0.025963,...,0.003805,-0.013601,-0.026617,0.026741,-0.077526,-0.032749,0.094584,0.004847,0.043765,0.001743
TRAIN_2474,-0.090639,0.025244,-0.054296,-0.027282,-0.086140,-0.005094,0.046781,0.043694,0.041948,-0.026018,...,-0.007459,-0.054953,0.020888,0.070636,-0.012312,-0.032366,0.027642,-0.049127,0.093051,-0.049553
TRAIN_2475,-0.044806,0.075822,-0.002418,-0.047819,-0.018801,0.059968,0.101083,0.114109,0.037637,0.055298,...,0.093023,0.041787,0.055687,-0.010128,-0.056995,0.019361,0.010142,0.035674,0.057293,0.014798


In [16]:
y = train_cleansed['first_party_winner']

In [17]:
y.info()

<class 'pandas.core.series.Series'>
Int64Index: 2478 entries, 0 to 2477
Series name: first_party_winner
Non-Null Count  Dtype
--------------  -----
2478 non-null   int64
dtypes: int64(1)
memory usage: 38.7 KB


In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1982, 384) (496, 384) (1982,) (496,)


In [17]:
def lgb_objective(trial):
    lgb_params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'dart',
        'application': 'multiclass',
        'max_depth': trial.suggest_int('max_depth', 4, 25),
        'num_boost_round': trial.suggest_int('num_boost_round', 100, 300),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 20),
        'n_estimators' : trial.suggest_int('n_estimators', 50, 2000),
        'num_leaves': trial.suggest_int('num_leaves', 10, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 0.8),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 0.8),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.01, 1.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.01, 1.0),
        'random_state': 42
    }
    lgb_model = lgb.LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train, y_train)
    lgb_preds = lgb_model.predict(X_val)
    return accuracy_score(y_val, lgb_preds)

In [33]:
def xgb_objective(trial):
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'n_estimators' : 39,
        'eta': 0.046266993101541545,
        'max_depth': 11,
        'subsample': 0.9668185485055184,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'colsample_bytree': 0.7588174046835052,
        'lambda': 23.696127246498573,
        'random_state': 42
    }
    


    xgb_model = xgb(**xgb_params)
    xgb_model.fit(X_train, y_train)
    xgb_preds = xgb_model.predict(X_val)
    return accuracy_score(y_val, xgb_preds)


In [27]:
def xgb_objective(trial):
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'n_estimators' : 39,
        'eta': 0.046266993101541545,
        'max_depth': 11,
        'subsample': 0.9668185485055184,
        'colsample_bytree': 0.7588174046835052,
        'lambda': 23.696127246498573,
        'random_state': 42
    }

In [19]:
lgb_study = optuna.create_study(direction='maximize')
lgb_study.optimize(lgb_objective, n_trials=1000)

[I 2023-06-06 16:48:46,212] A new study created in memory with name: no-name-c99e9a91-a0e6-46b3-84d2-3c0746513bbc


[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l1 is set=0.10438030150094596, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.10438030150094596
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] num_iterations is set=163, num_boost_round=163 will be ignored. Current value: num_iterations=163
[LightGBM] [Warning] lambda_l2 is set=0.25837187338424994, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.25837187338424994
[LightGBM] [Warning] feature_fraction is set=0.41153784833865553, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.41153784833865553
[LightGBM] [Warning] bagging_fraction is set=0.39623803810751823, subsample=1.0 will be ignored. Current value: bagging_fraction=0.39623803810751823


[I 2023-06-06 16:48:47,093] Trial 0 finished with value: 0.655241935483871 and parameters: {'max_depth': 7, 'num_boost_round': 163, 'min_data_in_leaf': 15, 'n_estimators': 1732, 'num_leaves': 1213, 'learning_rate': 0.024210046025646033, 'feature_fraction': 0.41153784833865553, 'bagging_fraction': 0.39623803810751823, 'bagging_freq': 4, 'lambda_l1': 0.10438030150094596, 'lambda_l2': 0.25837187338424994}. Best is trial 0 with value: 0.655241935483871.


[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] lambda_l1 is set=0.061522454515566614, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.061522454515566614
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] num_iterations is set=226, num_boost_round=226 will be ignored. Current value: num_iterations=226
[LightGBM] [Warning] lambda_l2 is set=0.016718761736309148, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.016718761736309148
[LightGBM] [Warning] feature_fraction is set=0.6664522188590496, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6664522188590496
[LightGBM] [Warning] bagging_fraction is set=0.6920484728356738, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6920484728356738


[I 2023-06-06 16:48:47,941] Trial 1 finished with value: 0.6612903225806451 and parameters: {'max_depth': 4, 'num_boost_round': 226, 'min_data_in_leaf': 20, 'n_estimators': 1709, 'num_leaves': 1123, 'learning_rate': 0.013437873448570878, 'feature_fraction': 0.6664522188590496, 'bagging_fraction': 0.6920484728356738, 'bagging_freq': 10, 'lambda_l1': 0.061522454515566614, 'lambda_l2': 0.016718761736309148}. Best is trial 1 with value: 0.6612903225806451.


...

[I 2023-06-06 16:52:17,715] Trial 197 finished with value: 0.6612903225806451 and parameters: {'max_depth': 8, 'num_boost_round': 100, 'min_data_in_leaf': 18, 'n_estimators': 1627, 'num_leaves': 687, 'learning_rate': 0.019042708907703026, 'feature_fraction': 0.5727629146229185, 'bagging_fraction': 0.751605883743882, 'bagging_freq': 6, 'lambda_l1': 0.3571118760520755, 'lambda_l2': 0.41703023271225204}. Best is trial 26 with value: 0.6733870967741935.


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] lambda_l1 is set=0.2318681269566261, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2318681269566261
[LightGBM] [Warning] min_data_in_leaf is set=12, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=12
[LightGBM] [Warning] num_iterations is set=106, num_boost_round=106 will be ignored. Current value: num_iterations=106
[LightGBM] [Warning] lambda_l2 is set=0.4654130569163095, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.4654130569163095
[LightGBM] [Warning] feature_fraction is set=0.5854816493451364, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5854816493451364
[LightGBM] [Warning] bagging_fraction is set=0.7680215235179763, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7680215235179763


[I 2023-06-06 16:52:18,920] Trial 198 finished with value: 0.6633064516129032 and parameters: {'max_depth': 8, 'num_boost_round': 106, 'min_data_in_leaf': 12, 'n_estimators': 872, 'num_leaves': 299, 'learning_rate': 0.03103774126399526, 'feature_fraction': 0.5854816493451364, 'bagging_fraction': 0.7680215235179763, 'bagging_freq': 7, 'lambda_l1': 0.2318681269566261, 'lambda_l2': 0.4654130569163095}. Best is trial 26 with value: 0.6733870967741935.


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l1 is set=0.1093463973463942, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1093463973463942
[LightGBM] [Warning] min_data_in_leaf is set=17, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=17
[LightGBM] [Warning] num_iterations is set=121, num_boost_round=121 will be ignored. Current value: num_iterations=121
[LightGBM] [Warning] lambda_l2 is set=0.6190838539635144, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6190838539635144
[LightGBM] [Warning] feature_fraction is set=0.6718128657187572, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6718128657187572
[LightGBM] [Warning] bagging_fraction is set=0.7176940252059597, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7176940252059597


[I 2023-06-06 16:52:20,159] Trial 199 finished with value: 0.6592741935483871 and parameters: {'max_depth': 8, 'num_boost_round': 121, 'min_data_in_leaf': 17, 'n_estimators': 1683, 'num_leaves': 406, 'learning_rate': 0.0351655052569127, 'feature_fraction': 0.6718128657187572, 'bagging_fraction': 0.7176940252059597, 'bagging_freq': 5, 'lambda_l1': 0.1093463973463942, 'lambda_l2': 0.6190838539635144}. Best is trial 26 with value: 0.6733870967741935.


In [ ]:
xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=1000)

[I 2023-06-08 17:15:53,027] A new study created in memory with name: no-name-0185a6bd-216c-4536-931b-c2de781ae2c8
[I 2023-06-08 17:15:54,208] Trial 0 finished with value: 0.6512096774193549 and parameters: {'learning_rate': 0.010588017398939697}. Best is trial 0 with value: 0.6512096774193549.
[I 2023-06-08 17:15:55,485] Trial 1 finished with value: 0.6491935483870968 and parameters: {'learning_rate': 0.030825192733751085}. Best is trial 0 with value: 0.6512096774193549.
[I 2023-06-08 17:15:56,730] Trial 2 finished with value: 0.6471774193548387 and parameters: {'learning_rate': 0.039736260680056694}. Best is trial 0 with value: 0.6512096774193549.
[I 2023-06-08 17:15:58,104] Trial 3 finished with value: 0.6532258064516129 and parameters: {'learning_rate': 0.06918879477070412}. Best is trial 3 with value: 0.6532258064516129.
[I 2023-06-08 17:15:59,369] Trial 4 finished with value: 0.6532258064516129 and parameters: {'learning_rate': 0.037970028005791745}. Best is trial 3 with value: 0.

[I 2023-06-08 17:16:56,939] Trial 45 finished with value: 0.655241935483871 and parameters: {'learning_rate': 0.018724827036447}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:16:58,257] Trial 46 finished with value: 0.655241935483871 and parameters: {'learning_rate': 0.025740244358613273}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:16:59,543] Trial 47 finished with value: 0.6491935483870968 and parameters: {'learning_rate': 0.030845706257229304}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:17:00,841] Trial 48 finished with value: 0.6532258064516129 and parameters: {'learning_rate': 0.017482488581293053}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:17:02,246] Trial 49 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.022586423549120674}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:17:03,903] Trial 50 finished with value: 0.655241935483871 and parameters:

[I 2023-06-08 17:17:54,745] Trial 90 finished with value: 0.6612903225806451 and parameters: {'learning_rate': 0.02062117401676394}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:17:55,989] Trial 91 finished with value: 0.6532258064516129 and parameters: {'learning_rate': 0.020627364575908372}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:17:57,203] Trial 92 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.02485733274608159}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:17:58,427] Trial 93 finished with value: 0.6612903225806451 and parameters: {'learning_rate': 0.022651876423867193}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:17:59,946] Trial 94 finished with value: 0.655241935483871 and parameters: {'learning_rate': 0.022071722586844072}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:18:01,290] Trial 95 finished with value: 0.655241935483871 and paramete

[I 2023-06-08 17:18:53,650] Trial 135 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.024142457517138335}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:18:54,979] Trial 136 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.02758929662390963}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:18:56,273] Trial 137 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.024864630471226043}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:18:57,587] Trial 138 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.022529120409915986}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:18:58,889] Trial 139 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.023469334042766625}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:19:00,191] Trial 140 finished with value: 0.6512096774193549 and p

[I 2023-06-08 17:19:54,564] Trial 180 finished with value: 0.655241935483871 and parameters: {'learning_rate': 0.02575042945450494}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:19:55,935] Trial 181 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.02507343207413673}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:19:57,229] Trial 182 finished with value: 0.6612903225806451 and parameters: {'learning_rate': 0.022958632018006728}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:19:58,787] Trial 183 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.022791174686829892}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:20:00,132] Trial 184 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.02387267282763984}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:20:01,413] Trial 185 finished with value: 0.6532258064516129 and par

[I 2023-06-08 17:20:52,586] Trial 225 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.023298386219486007}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:20:53,806] Trial 226 finished with value: 0.6391129032258065 and parameters: {'learning_rate': 0.060381916533019914}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:20:54,949] Trial 227 finished with value: 0.6512096774193549 and parameters: {'learning_rate': 0.011935598680409562}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:20:56,232] Trial 228 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.025488879879869156}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:20:57,577] Trial 229 finished with value: 0.655241935483871 and parameters: {'learning_rate': 0.021108532804546297}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:20:58,854] Trial 230 finished with value: 0.6532258064516129 and

[I 2023-06-08 17:21:52,105] Trial 270 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.025586953310370714}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:21:53,337] Trial 271 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.04987472434817593}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:21:54,647] Trial 272 finished with value: 0.6532258064516129 and parameters: {'learning_rate': 0.021412717610955555}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:21:55,964] Trial 273 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.023025372423739685}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:21:57,300] Trial 274 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.027426244549512667}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:21:58,638] Trial 275 finished with value: 0.6592741935483871 and pa

[I 2023-06-08 17:22:50,561] Trial 315 finished with value: 0.6532258064516129 and parameters: {'learning_rate': 0.026719095567868743}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:22:51,828] Trial 316 finished with value: 0.6592741935483871 and parameters: {'learning_rate': 0.024090788387655097}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:22:53,102] Trial 317 finished with value: 0.6532258064516129 and parameters: {'learning_rate': 0.021035705351514482}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:22:54,354] Trial 318 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.02250182183985659}. Best is trial 40 with value: 0.6612903225806451.
[I 2023-06-08 17:22:55,585] Trial 319 finished with value: 0.657258064516129 and parameters: {'learning_rate': 0.02555459337052296}. Best is trial 40 with value: 0.6612903225806451.


In [28]:
print('Number of finished LGB trials: {}'.format(len(lgb_study.trials)))
print('LGB Best trial:')
lgb_trial = lgb_study.best_trial

print('  Value: {}'.format(lgb_trial.value))
print('  Params: ')

for key, value in lgb_trial.params.items():
    print('    {}: {}'.format(key, value))


Number of finished LGB trials: 200
LGB Best trial:
  Value: 0.6733870967741935
  Params: 
    max_depth: 8
    num_boost_round: 153
    min_data_in_leaf: 17
    n_estimators: 934
    num_leaves: 290
    learning_rate: 0.03303130202695269
    feature_fraction: 0.5996426902668317
    bagging_fraction: 0.7298975471329071
    bagging_freq: 3
    lambda_l1: 0.3444854238234955
    lambda_l2: 0.32190717279983505


In [29]:
print('Number of finished XGB trials: {}'.format(len(xgb_study.trials)))
print('XGB Best trial:')
xgb_trial = xgb_study.best_trial

print('  Value: {}'.format(xgb_trial.value))
print('  Params: ')

for key, value in xgb_trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished XGB trials: 1000
XGB Best trial:
  Value: 0.6814516129032258
  Params: 
    n_estimators: 39
    eta: 0.046266993101541545
    max_depth: 11
    subsample: 0.9668185485055184
    colsample_bytree: 0.7588174046835052
    lambda: 23.696127246498573


In [42]:
lgb_best_params = lgb_study.best_params
lgb_best_params['random_state'] = 42
lgb_model = lgb.LGBMClassifier(**lgb_best_params)
lgb_model.fit(X_train, y_train)
lgb_preds = lgb_model.predict(X_val)
lgb_accuracy = accuracy_score(y_val, lgb_preds)


In [34]:
xgb_params = xgb_trial.params
XGB = xgb(**xgb_params)
XGB.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7588174046835052, early_stopping_rounds=None,
              enable_categorical=False, eta=0.046266993101541545,
              eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=23.696127246498573,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=11,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=39, n_jobs=None,
              num_parallel_tree=None, ...)

In [40]:
LGB_pred = lgb.LGBMClassifier.predict(X_val)

TypeError: LGBMClassifier.predict() missing 1 required positional argument: 'X'

In [41]:
XGB_pred = XGB.predict(X_val)

In [43]:
accuracy = accuracy_score(y_val, lgb_preds)
print("Accuracy after tuning: %.2f%%" % (accuracy * 100.0))
accuracy = accuracy_score(y_val, XGB_pred)
print("\nAccuracy after tuning: %.2f%%" % (accuracy * 100.0))

Accuracy after tuning: 65.73%

Accuracy after tuning: 66.53%


In [44]:
print("-- LGB --")
print("Train ACC : %.3f" % accuracy_score(y_train, lgb_model.predict(X_train)))
print("Val ACC : %.3f" % accuracy_score(y_val, lgb_model.predict(X_val)))

print("\n-- XGB --")
print("Train ACC : %.3f" % accuracy_score(y_train, XGB.predict(X_train)))
print("Val ACC : %.3f" % accuracy_score(y_val, XGB.predict(X_val)))

-- LGB --
Train ACC : 1.000
Val ACC : 0.657

-- XGB --
Train ACC : 0.894
Val ACC : 0.665


In [45]:
print(classification_report(y_val, lgb_preds))
print(classification_report(y_val, XGB_pred))

              precision    recall  f1-score   support

           0       0.45      0.06      0.11       168
           1       0.67      0.96      0.79       328

    accuracy                           0.66       496
   macro avg       0.56      0.51      0.45       496
weighted avg       0.59      0.66      0.56       496

              precision    recall  f1-score   support

           0       0.56      0.05      0.10       168
           1       0.67      0.98      0.79       328

    accuracy                           0.67       496
   macro avg       0.62      0.52      0.45       496
weighted avg       0.63      0.67      0.56       496



In [46]:
LGB_submission = pd.read_csv('./sample_submission.csv')
LGB_pred = pd.DataFrame(lgb_preds.astype(int))
LGB_submission['first_party_winner'] = LGB_pred
LGB_submission.to_csv("./LGB_tryout_submission.csv", index=False)

XGB_submission = pd.read_csv('./sample_submission.csv')
XGB_pred = pd.DataFrame(XGB_pred.astype(int))
XGB_submission['first_party_winner'] = XGB_pred
XGB_submission.to_csv("./XGB_tryout_submission.csv", index=False)

In [ ]:
submission = pd.DataFrame_csv('../')